# Programming with Python

## Lecture 06: Type hints, descriptors

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 22 Mar, 2025

## TypedDict

`TypedDict` declares a dictionary type that expects all of its instances to have a certain set of keys, where each key is associated with a value of a consistent type. This expectation is not checked at runtime but is only enforced by type checkers.

It’s tempting to use TypedDict to protect against errors while handling dynamic data structures like JSON API responses. But correct handling of JSON must be done at runtime, and not with static type checking. For runtime
checking of JSON-like structures using type hints, check out the `pydantic` package.

*References:*

- [TypedDict](https://docs.python.org/3/library/typing.html#typing.TypedDict)
- Fluent Python, Luciano Ramalho

### Practice

Show example 1.

## Type casting

`typing.cast` is used to explicitly tell the type checker that an expression should be treated as a specific type. It does not change the actual value at runtime but is useful for static type checking with tools like `mypy`.

### Practice

Show example 2.

# Generic

**Generic classes** allow defining reusable classes that can handle multiple types while maintaining type safety. They can be defined by using `typing.Generic` abstract base class for generic types.

### Practice

Show example 3.

## Generic static protocols

**Generic static protocols** allow defining interfaces that enforce structure while keeping type flexibility. They are useful when designing generic utilities, factories or contracts.

`typing` module includes several generic static protocols, such as `SupportsAbs`, `SupportsInt`, `SupportsFloat`.

## `typing.runtime_checkable`

The `@runtime_checkable` decorator from `typing` allows protocols to be checked at runtime using `isinstance()` and `issubclass()`. This is useful when working with structural checks similar to abstract base classes.

### Practice

Show example 4.

## Typing map

![Typing map](resources/typing_map.png)

## Descriptors

**Descriptors** are Python objects that implement a method of the descriptor protocol that define the behavior of attribute access. They allow you to customize how attributes are retrieved, set, or deleted. Descriptors are a key part of Python’s object model and are used behind the scenes in properties, methods, static methods, class methods and `super()`. Additionally, their use cases include validation, read-only, computed and lazy attributes.

The default behavior for attribute access is to get, set, or delete the attribute from an object’s dictionary. For instance, `a.x` has a lookup chain starting with `a.__dict__['x']`, then `type(a).__dict__['x']`, and continuing through the method resolution order of `type(a)`. If the looked-up value is an object defining one of the descriptor methods, then Python may override the default behavior and invoke the descriptor method instead. Where this occurs in the precedence chain depends on which descriptor methods were defined.

### Descriptor protocol

The definition of the **descriptor protocol** includes the following methods:

```python
__get__(self, instance, owner=None) -> object
__set__(self, instance, value) -> None
__delete__(self, instance) -> None
__set_name__(self, owner, name)
```

- `self` is the instance of the descriptor.
- `instance` is the instance of the object the descriptor is attached to.
- `owner` is the type of the object the descriptor is attached to.

Descriptors can be classified into:

- **Non-data descriptors:** Only implement `__get__` (e.g. methods).
- **Data descriptors:** Implement `__set__` or `__delete__` along with `__get__` (e.g. properties).

*References*

Throughtout this section the following resources are heavily used:

- [Descriptor Guide](https://docs.python.org/3/howto/descriptor.html)

In [ ]:
class SimpleDescriptor:
    def __init__(self, name=None):
        self.name = name

    def __get__(self, instance, owner=None):
        print(f"__get__ called: self={self}, instance={instance}, owner={owner}")
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None)

    def __set__(self, instance, value):
        print(f"__set__ called: self={self}, instance={instance}, value={value}")
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        print(f"__delete__ called: self={self}, instance={instance}")
        del instance.__dict__[self.name]

class Point:
    x = SimpleDescriptor("x")
    y = SimpleDescriptor("y")

In [ ]:
obj = Point()

In [ ]:
obj.x = 10 # Triggers __set__

In [ ]:
obj.x # Triggers __get__

In [ ]:
del obj.x # Triggers __delete__

In [ ]:
Point.x # Triggers __get__ with instance set to None

### Read-only data descriptor

A **read-only data descriptor** is a descriptor that allows attribute access but prevents modification. It is commonly used to enforce immutability in class attributes.

Read-only data descritor is defined in the following way:

- Implements `__get__()` to return a value.
- Implements `__set__()` but raises an error to prevent modification.
- Optionally, implements `__delete__()` to prevent deletion.

In [ ]:
class ReadOnly:
    def __init__(self, name=None):
        self.name = name

    def __get__(self, instance, owner=None):
        print(f"__get__ called: self={self}, instance={instance}, owner={owner}")
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None) # Return stored value

    def __set__(self, instance, value):
        raise AttributeError("This attribute is read-only") # Prevent modification

    def __delete__(self, instance):
        raise AttributeError("Cannot delete this attribute") # Prevent deletion

class MyClass:
    attr = ReadOnly("attr")  # Read-only attribute

In [ ]:
obj = MyClass()
obj.attr

In [ ]:
try:
    obj.attr = 100  # Should raise an error
except AttributeError as e:
    print(e)

In [ ]:
try:
    del obj.attr  # Should raise an error
except AttributeError as e:
    print(e)

### `__dict__` attribute

The `__dict__` attribute in Python is a dictionary that stores an object's writable attributes. It is available for instances and classes that allow dynamic attributes.

In [ ]:
class Rectangle:
    num_of_sides: int = 4
    
    def __init__(self, width: int, height: int) -> None:
        self.width = width
        self.height = height

In [ ]:
r = Rectangle(1, 2)

print(r.__dict__)
print(type(r))
print(type(r).__dict__) # Same as print(Rectangle.__dict__)

In [ ]:
print(r.width)
print(type(r).num_of_sides)

In [ ]:
# Using __dict__ attribute

print(r.__dict__["width"])
print(type(r).__dict__["num_of_sides"])

### Descriptor invocation

A descriptor can be called directly with `desc.__get__(obj)` or `desc.__get__(None, cls)`.

But it is more common for a descriptor to be invoked automatically from attribute access.

Descriptors are invoked by the [`__getattribute__(self, name)`](https://docs.python.org/3/reference/datamodel.html#object.__getattribute__) method, which returns the attribute value or raises an `AttributeError` exception if an attribute is not found.

The expression `obj.x` looks up the attribute `x` in the chain of namespaces for `obj`. If the search finds a descriptor outside of the instance `__dict__`, its `__get__()` method is invoked according to the precedence rules listed below.

1. **`__getattribute__` method:** First the object's `__getattribute__` method is called, which is responsible for attribute access.
2. **Data Descriptors:** If the attribute is found in the class (or its parent classes) and is a data descriptor, the descriptor's `__get__` method is called.
3. **Instance Dictionary:** If the attribute is found in the object's `__dict__`, that value is returned.
4. **Non-Data Descriptors**: If the attribute is found in the class (or its parent classes) and is a non-data descriptor (implements only `__get__`), the descriptor's `__get__` method is called.
5. **Class Dictionary:** If the attribute is found in the class's `__dict__` (or its parent classes), that value is returned.
6. **`__getattr__` method:** If the attribute is not found anywhere else and the object has a `__getattr__` method, that method is called.
7. **`AttributeError`:** If all the above steps fail, Python raises an `AttributeError`.

In [ ]:
class DataDescriptor:
    """A descriptor that implements both __get__ and __set__"""
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, owner=None):
        print(f"2. DataDescriptor.__get__ called for {self.name}")
        return f"DataDescriptor value for {self.name}"
        
    def __set__(self, instance, value):
        print(f"DataDescriptor.__set__ called for {self.name} with value {value}")

class NonDataDescriptor:
    """A descriptor that implements only __get__"""
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, owner=None):
        print(f"4. NonDataDescriptor.__get__ called for {self.name}")
        return f"NonDataDescriptor value for {self.name}"
    
class OverridingNoGetDescriptor:
    """A descriptor that implements only __set__"""
    def __init__(self, name):
        self.name = name
        
    def __set__(self, instance, value):
        print(f"OverridingNoGetDescriptor.__set__ called for {self.name} with value {value}")

class MyClass:
    data_desc_attr = DataDescriptor("data_desc") # Data descriptor
    non_data_desc_attr = NonDataDescriptor("non_data_desc") # Non-data descriptor
    over_no_get_attr = OverridingNoGetDescriptor("overriding_no_get_attr") # Overriding descriptor with no __get__
    class_attr = "class attribute" # Regular class attribute
    
    def __init__(self):
        self.instance_attr = "instance attribute" # Regular instance attribute
        
    def __getattr__(self, name):
        print(f"6. __getattr__ called for {name}")
        return f"__getattr__ value for {name}"
        
    def __getattribute__(self, name):
        print(f"1. __getattribute__ called for {name}")
        return super().__getattribute__(name)

#### Lookup chain for different attributes

In [ ]:
obj = MyClass()

print("Accessing data descriptor:")
print(obj.data_desc_attr)

print("\nAccessing instance attribute:")
print(obj.instance_attr)

print("\nAccessing non-data descriptor:")
print(obj.non_data_desc_attr)

print("\nAccessing class attribute:")
print(obj.class_attr)

print("\nAccessing non-existent attribute:")
print(obj.doesnt_exist_attr)

print("\nAccessing overriding wiht no __get__ descriptor:")
print(obj.over_no_get_attr)

#### Instance attributes override non-data descriptors

In [ ]:
obj = MyClass()

obj.__dict__["non_data_desc_attr"] = "instance value overriding non-data descriptor"

print("\nAccessing non-data descriptor after adding instance attribute:")
print(obj.non_data_desc_attr)

#### Data descriptors override instance attributes

In [ ]:
obj = MyClass()

obj.__dict__["data_desc_attr"] = "instance value trying to override data descriptor"

print("\nAccessing data descriptor after adding instance attribute:")
print(obj.data_desc_attr)

#### Instance attributes override "Overriding descriptor with no `__get__`" descriptors

In [ ]:
obj = MyClass()

obj.__dict__["over_no_get_attr"] = "instance value overriding \"Overriding descriptor with no __get__\" descriptor"

print("\nAccessing overriding wiht no __get__ descriptor after adding instance attribute:")
print(obj.over_no_get_attr)

### `__set_name__` magic method

The `__set_name__(self, owner, name)` method is a special method for descriptors. It is automatically called when a descriptor is assigned as a class attribute that defines the name of the attribute the descriptor is managing. Thus, we do not need to manually set the attribute name.

In [ ]:
class SimpleDescriptor:
    def __set_name__(self, owner, name):
        print(f"__set_name__ called: owner={owner}, name={name}")
        self.name = name  # Store attribute name

    def __get__(self, instance, owner):
        return instance.__dict__.get(self.name, None)

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

class MyClass:
    attr = SimpleDescriptor()  # Triggers __set_name__

In [ ]:
obj = MyClass()

obj.attr = 42
obj.attr

### Validation example

The `Validator` class is both an abstract base class and a managed attribute descriptor. Custom validators need to inherit from `Validator` and must supply a `validate()` method.

In [ ]:
from abc import ABC, abstractmethod

class Validator(ABC):
    """Base descriptor class for validation"""

    def __set_name__(self, owner, name):
        self.name = name
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass

In [ ]:
class StringValidator(Validator):
    """Validates string values with optional length constraints"""

    def __init__(self, min_length=None, max_length=None):
        self.min_length = min_length
        self.max_length = max_length

    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f"{self.name} must be a string")
        
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f"{self.name} must be at least {self.min_length} characters long")
        
        if self.max_length is not None and len(value) > self.max_length:
            raise ValueError(f"{self.name} must be no more than {self.max_length} characters long")
        
        return value

In [ ]:
class RangeValidator(Validator):
    """Validates that a numeric value is within a specified range"""

    def __init__(self, minimum=None, maximum=None):
        self.minimum = minimum
        self.maximum = maximum
    
    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f"{self.name} must be a number")
        
        if self.minimum is not None and value < self.minimum:
            raise ValueError(f"{self.name} must be at least {self.minimum}")
        
        if self.maximum is not None and value > self.maximum:
            raise ValueError(f"{self.name} must be no more than {self.maximum}")
        
        return value

In [ ]:
class Person:
    name = StringValidator(min_length=2, max_length=50)
    age = RangeValidator(minimum=0, maximum=150)
    
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __repr__(self):
        return f"Person(name={self.name}, age={self.age})"

In [ ]:
try:
    Person("J", 42)
except ValueError as e:
    print(e)

In [ ]:
try:
    Person("John Doe", -42)
except ValueError as e:
    print(e)

In [ ]:
Person("John Doe", 42)

### LazyProperty example

A lazy evaluation of a property is a design pattern used in programming where a property of an object is computed only when it is first accessed, and the result is then cached for future accesses. This can improve performance by delaying expensive computations until they are actually needed.

We can use *non-data* descriptor to implement lazy property.

In [ ]:
import time

class LazyProperty:
    def __init__(self, func):
        self.func = func
        self.name = func.__name__

    def __get__(self, instance, owner):
        if instance is None:
            return self
        value = self.func(instance)
        setattr(instance, self.name, value)  # Cache the computed value
        return value

class Model:
    @LazyProperty
    def expensive_value(self):
        time.sleep(5)
        return 42

In [ ]:
m = Model()

In [ ]:
m.expensive_value # computed on the first access

In [ ]:
m.expensive_value # cached value is returned

## Descriptors in Python internals

### Descriptors in properties

`property()` is implemented in terms of the descriptor protocol. `property()` returns a `Property` object that implements the descriptor protocol. It uses the parameters `fget`, `fset` and `fdel` for the actual implementation of the three methods of the protocol.

Here is a pure Python equivalent that implements most of the core functionality.

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __set_name__(self, owner, name):
        self.__name__ = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

### Descriptors in functions and methods

Python’s object oriented features are built upon a function based environment. Using non-data descriptors, the two are merged seamlessly.

Functions stored in class dictionaries get turned into methods when invoked. Methods only differ from regular functions in that the object instance is prepended to the other arguments. By convention, the instance is called self but could be called this or any other variable name.

Methods can be created manually with `types.MethodType` which is roughly equivalent to the following class in Python.

In [ ]:
class MethodType:
    "Emulate PyMethod_Type in Objects/classobject.c"

    def __init__(self, func, obj):
        self.__func__ = func
        self.__self__ = obj

    def __call__(self, *args, **kwargs):
        func = self.__func__
        obj = self.__self__
        return func(obj, *args, **kwargs)

    def __getattribute__(self, name):
        "Emulate method_getset() in Objects/classobject.c"
        if name == '__doc__':
            return self.__func__.__doc__
        return object.__getattribute__(self, name)

    def __getattr__(self, name):
        "Emulate method_getattro() in Objects/classobject.c"
        return getattr(self.__func__, name)

    def __get__(self, obj, objtype=None):
        "Emulate method_descr_get() in Objects/classobject.c"
        return self

To support automatic creation of methods, functions include the `__get__()` method for binding methods during attribute access. This means that functions are non-data descriptors that return bound methods during dotted lookup from an instance.

The following code shows how this works.

In [ ]:
class Function:
    ...

    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        if obj is None:
            return self
        return MethodType(self, obj)

#### How the function descriptor works in practice

In [ ]:
class MyClass:
    def my_func(self):
         return self

Accessing the function through the class dictionary or dotted access from a class does not invoke `__get__()`. Instead, it just returns the underlying function object.

In [ ]:
MyClass.__dict__["my_func"]

In [ ]:
MyClass.my_func

The dotted lookup from an instance calls `__get__()` which returns a bound method object.

In [ ]:
obj = MyClass()
obj.my_func

Internally, the bound method stores the underlying function and the bound instance.

In [ ]:
obj.my_func.__func__

In [ ]:
obj.my_func.__self__

In [ ]:
obj is obj.my_func.__self__

This is why `self` variable name is commonly used in methods.

### Method binding

Non-data descriptors provide a simple mechanism for variations on the usual patterns of binding functions into methods.

This chart summarizes the binding and its two most useful variants:


| Transformation | Called from an object | Called from a class |
|---|---|---|
| function | f(obj, *args) | f(*args) |
| staticmethod | f(*args) | f(*args) |
| classmethod | f(type(obj), *args) | f(cls, *args) |

### Static methods

Using the non-data descriptor protocol, a pure Python version of `staticmethod()` would look like the following.

In [ ]:
class StaticMethod:
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f

    def __call__(self, *args, **kwds):
        return self.f(*args, **kwds)

### Class methods

Using the non-data descriptor protocol, a pure Python version of classmethod() would look like the following.

In [ ]:
class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"
    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        def newfunc(*args):
            return self.f(klass, *args)
        return newfunc